# Theta Sketch Tutorial


## Table of Contents

  * [Overview](#Overview)
  * [Set-up](#Set-up)
  * [Basic Sketch Usage](#Basic-Sketch-Usage)
  * [Sketch Unions](#Sketch-Unions)
  * [Sketch Intersections](#Sketch-Intersections)
  * [Set Difference (A-not-B)](#Set-Difference-(A-not-B))

### Overview

This tutorial covers basic operation of the Theta sketch for distinct counting. We will demonstrate how to create and feed data into sketches as well as the various set operations. We will also include the HLL sketch for comparison.

Characterization tests of the hash function we use, Murmur3, have shown that it has excellent independence properties. As a reuslt, we can achieve reasonable performance for demonstration purposes by feeding in sequential integers. This lets us experiment with the set operations in a controlled but still realistic manner, and to know the exact result without resorting to an expensive computation.

### Set-up

This tutorial assuems you have already downloaded and installed the python wrapper for the DataSketches library. See the [DataSketches Downloads](http://datasketches.apache.org/docs/Community/Downloads.html) page for details

In [ ]:
from datasketches import theta_sketch, update_theta_sketch, compact_theta_sketch
from datasketches import theta_union, theta_intersection, theta_a_not_b

from datasketches import hll_sketch, hll_union

### Basic Sketch Usage

To start, we'll create a sketch with ~1 million points in order to demonstrate basic sketch operations.

In [ ]:
n = 2**20
k = 12
sk1 = update_theta_sketch(k)
hll1 = hll_sketch(k)
for i in range(0, n):
    sk1.update(i)
    hll1.update(i)
print(sk1)
print(hll1)

The summary contains most data of interest, but we can also query for specific information. And in this case, since we know the exact number of distinct items presented to the sketch, we can look at the estimate, upper, and lower bounds as a percentage of the exact value.

In [ ]:
print(f'Exact result:\t\t\t{n}')
print('')
print(f'Theta upper bound (1 std. dev):\t{sk1.get_upper_bound(1):.1f}\t({100*sk1.get_upper_bound(1) / n - 100:.2f}%)')
print(f'Theta estimate:\t\t\t{sk1.get_estimate():.1f}\t({100*sk1.get_estimate() / n - 100:.2f}%)')
print(f'Theta lower bound (1 std. dev):\t{sk1.get_lower_bound(1):.1f}\t({100*sk1.get_lower_bound(1) / n - 100:.2f}%)')
print('')
print(f'HLL upper bound (1 std. dev):\t{hll1.get_upper_bound(1):.1f}\t({100*hll1.get_upper_bound(1) / n - 100:.2f}%)')
print(f'HLL estimate:\t\t\t{hll1.get_estimate():.1f}\t({100*hll1.get_estimate() / n - 100:.2f}%)')
print(f'HLL lower bound (1 std. dev):\t{hll1.get_lower_bound(1):.1f}\t({100*hll1.get_lower_bound(1) / n - 100:.2f}%)')


We can serialize and reconstruct the sketch. If we compact the sketch prior to serialization, we can still query the rebuilt sketch but cannot update it further. When reconstructed, we can see that the estimate is exactly the same.

In [ ]:
sk1_bytes = sk1.compact().serialize()
len(sk1_bytes)

In [ ]:
new_sk1 = theta_sketch.deserialize(sk1_bytes)
print(f'Estimate (original):\t{sk1.get_estimate()}')
print(f'Estimate (new):\t\t{new_sk1.get_estimate()}')

### Sketch Unions

Theta Sketch unions make use of a separate union object. The union will accept input sketches with different values of $k$.

For this example, we will create a sketch with distinct values that partially overlap those in `sk1`.

In [ ]:
offset = int(15 * n / 16)
sk2 = update_theta_sketch(k+1)
hll2 = hll_sketch(k+1)
for i in range(0, n):
    sk2.update(i + offset)
    hll2.update(i + offset)
print(sk2)

We can now feed the sketches into the union. As constructed, the exact number of unique values presented to the two sketches is $\frac{31}{16}n$.

In [ ]:
union = theta_union(k)
union.update(sk1)
union.update(sk2)

union_hll = hll_union(k)
union_hll.update(hll1)
union_hll.update(hll2)

exact = int(31 * n / 16);
result = union.get_result()
theta_bound_pct = 100 * (result.get_upper_bound(1) - result.get_estimate()) / exact

hll_result = union_hll.get_result()
hll_bound_pct = 100 * (hll_result.get_upper_bound(1) - hll_result.get_estimate()) / exact


print(f'Exact result:\t{exact}')
print(f'Theta Estimate:\t{result.get_estimate():.1f} ({100*(result.get_estimate()/exact - 1):.2f}% +- {theta_bound_pct:.2f}%)')
print(f'HLL Estimate:\t{hll_result.get_estimate():.1f} ({100*(hll_result.get_estimate()/exact - 1):.2f}% +- {hll_bound_pct:.2f}%)')


### Sketch Intersections

Beyond unions, theta sketches also support intersctions through the use of an intersection object. For comparison, we also present the HLL estimate here using, using the Inclusion-Exclusion formula: $|A \cup B| = |A| + |B| - |A \cap B|$.

That formula might not seem too bad when intersecting 2 sketches, but as the number of sketches increases the formula becomes increasingly comples, and the error compounds rapidly. By comparison, the Theta set operations can be applied to an arbitrary number of sketches. 

In [ ]:
intersection = theta_intersection()
intersection.update(sk1)
intersection.update(sk2)

hll_inter_est = hll1.get_estimate() + hll2.get_estimate() - hll_result.get_estimate()

print("Has result: ", intersection.has_result())
result = intersection.get_result()
print(result)

In this case, we expect the sets to have an overlap of $\frac{1}{16}n$.

In [ ]:
exact = int(n / 16)
theta_bound_pct = 100 * (result.get_upper_bound(1) - result.get_estimate()) / exact

print(f'Exact result:\t\t{exact}')
print(f'Theta Estimate:\t\t{result.get_estimate():.1f} ({100*(result.get_estimate()/exact - 1):.2f}% +- {theta_bound_pct:.2f}%)')
print(f'HLL Estimate:\t\t{hll_inter_est:.1f} ({100*(hll_inter_est/exact - 1):.2f}%)')

### Set Difference (A-not-B)

Finally, we have the set difference operation. Unlike `theta_union` and `theta_intersection`, `theta_a_not_b` is currently stateless: The object takes as input 2 sketches at a time, namely $a$ and $b$, and directly returns the result as a sketch.

In [ ]:
anb = theta_a_not_b()
result = anb.compute(sk1, sk2)
print(result)

By using the same two sketches as before, the expected result here is $\frac{15}{16}n$.

Our HLL estimate comes from manipulating the Inclusion-Exclusion formula above to obtain $|A| - |A \cap B| = |A \cup B| - |B|$.

In [ ]:
exact = int(15 * n /16)
theta_bound_pct = 100 * (result.get_upper_bound(1) - result.get_estimate()) / exact
hll_diff_est = hll_result.get_estimate() - hll2.get_estimate()

print(f'Exact result:\t{exact}')
print(f'Theta estimate:\t{result.get_estimate():.1f} ({100*(result.get_estimate()/exact -1):.2f}% +- {theta_bound_pct:.2f}%)')
print(f'HLL estimate:\t{hll_diff_est:.1f} ({100*(hll_diff_est/exact - 1):.2f}%)')